# Aula 5 - pandas

Na aula de hoje, vamos explorar os seguintes tópicos em Python:

- 1) Criar tabelas dinâmicas e fazer reshape do df (Melt, pivot, pivot_table)
- 2) Transformação de Dados (cut, qcut, get_dummies)
- 3) Utilidades Extras (multiindex to singleindex, combine_first)
_______

### Objetivos

Apresentar como criar tabelas dinâmicas, como fazer transformações em dados contínuos e categóricos e aprender como trabalhar com multiindex

____
____
____

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

In [13]:
df = pd.read_csv("titanic.csv")

In [14]:
df.drop(['PassengerId','Ticket','Name'],inplace=True,axis=1)

## Construindo uma Tabela Dinâmica usando Pandas

É hora de construir uma tabela dinâmica em Python usando a incrível biblioteca Pandas! Exploraremos as diferentes facetas de uma tabela dinâmica neste artigo e construiremos uma tabela dinâmica incrível e flexível a partir do zero.


    * pivot_table requer um dado e um parâmetro de índice
    * data é o dataframe do Pandas que você passa para a função
    * índice é o recurso que permite agrupar seus dados. O recurso de índice aparecerá como um índice na tabela resultante



In [15]:
# index único
table = pd.pivot_table(data=df, index=['Sex'])
table

,Age,Fare,Parch,Pclass,SibSp,Survived
Sex,,,,,,
female,27.915709,44.479818,0.649682,2.159236,0.694268,0.742038
male,30.726645,25.523893,0.235702,2.389948,0.429809,0.188908


In [6]:
# múltiplos indexes
table = pd.pivot_table(df, index=['Sex','Pclass'])
table

Age        Fare     Parch     SibSp  Survived
Sex    Pclass                                                     
female 1       34.611765  106.125798  0.457447  0.553191  0.968085
       2       28.722973   21.970121  0.605263  0.486842  0.921053
       3       21.750000   16.118810  0.798611  0.895833  0.500000
male   1       41.281386   67.226127  0.278689  0.311475  0.368852
       2       30.740707   19.741782  0.222222  0.342593  0.157407
       3       26.507589   12.661633  0.224784  0.498559  0.135447

### Função de agregação
Por padrão o `.pivot_table()` utiliza o `np.mean()` como função de agragação, mas podemos utilizar diferentes funções de agregação para diferentes colunas. Para isso, precisamos de um dicionário como entrada para o parâmetro aggfunc com o nome da coluna como chave e a função agregada como o valor. <br>
Vamos criar uma pivot table calculando a média de 'Age' e a soma para o 'Survived':


In [7]:
# diferentes funções de agregação
table = pd.pivot_table(df, 
                       index=['Sex','Pclass'], 
                       aggfunc={'Age':np.mean, 'Survived':np.sum})
table

Age  Survived
Sex    Pclass                     
female 1       34.611765        91
       2       28.722973        70
       3       21.750000        72
male   1       41.281386        45
       2       30.740707        17
       3       26.507589        47

Qual a diferença entre esse pivot_table e um groupby?

In [19]:
df.groupby(['Sex','Pclass']).agg({'Age':"mean",'Survived':sum})

Age  Survived
Sex    Pclass                     
female 1       34.611765        91
       2       28.722973        70
       3       21.750000        72
male   1       41.281386        45
       2       30.740707        17
       3       26.507589        47

Para ficar mais parecido à tabela dinâmica do excel podemos indicar um dos index para ser visualizado como coluna e adicionar os totais de colunas e índices:

In [23]:
table = pd.pivot_table(df,
                       index=['Sex'],
                       columns=['Pclass'],
                       values=['Survived'],
                       aggfunc=np.sum,
                       margins=1)
table

Survived              
Pclass        1   2    3  All
Sex                          
female       91  70   72  233
male         45  17   47  109
All         136  87  119  342

E se quisessemos a porcentagem de cada um? Podemos calcular a contagem total de cada grupo:

In [18]:
table2 = pd.pivot_table(df,
                       index=['Sex'],
                       columns=['Pclass'],
                       values=['Survived'],
                       aggfunc='count',
                       margins=1)

In [19]:
table2

Survived               
Pclass        1    2    3  All
Sex                           
female       94   76  144  314
male        122  108  347  577
All         216  184  491  891

e dividir um df pelo outro. Essa divisão será feita elemento a elemento.

In [63]:
table_prob = table/table2
table_prob

Survived                              
Pclass         1         2         3       All
Sex                                           
female  0.968085  0.921053  0.500000  0.742038
male    0.368852  0.157407  0.135447  0.188908
All     0.629630  0.472826  0.242363  0.383838

Formatando nossa saída

In [37]:
(table_prob*100).style.format('{0:,.1f}%')

O `pd.pivot_table()` nos permite passar vários parâmetros úteis: <br>
pandas.pivot_table(data, values=None, index=None, columns=None, aggfunc='mean', fill_value=None, margins=False, dropna=True, margins_name='All', observed=False, sort=True) <br>
Segue a mesma ideia de `pd.unstack()`.

## Desfazendo uma tabela dinâmica
Para fazer um unpivoting utilizamos o `pd.melt()`. Esse método é utilizado quando queremos que uma ou mais colunas se tornem colunas de identificadores. Segue a mesma ideia de `pd.stack()`. As colunas que vamos dissolver são definidas por `id_vars` e `value_vars`.

Parâmetros: <br>
pandas.melt(frame, id_vars=None, value_vars=None, var_name=None, value_name='value', col_level=None, ignore_index=True)


   * __frame__: DataFrame <br>
   * __id_vars__: Colunas para serem usadas como categorias. São as colunas que você quer manter do jeito que estão. <br> 
   * __value_vars__: Colunas para o unpivot - para sairem do formato largo para longo (wide to long). Se não especificada, usa todas as colunas que não estão em id_vars. <br> 
   * __var_name__: Nome para a nova coluna de variáveis categóricas. <br>
   * __value_name__: Nome para ser utilizado na coluna de valores. <br>
   * __col_level__: Se as colunas são MultiIndex.<br>

Vamos simplificar nossa tabela de probabilidades eliminando a coluna e linha com os totais, resetando o index e eliminando o multi-index do nome das colunas renomeando-as.

In [61]:
table_prob = table_prob.drop(('Survived', 'All'), axis=1).drop('All').reset_index()
table_prob.columns = ['Sex', 1, 2, 3]
table_prob

,Sex,1,2,3
0,female,0.968085,0.921053,0.500000
1,male,0.368852,0.157407,0.135447


Observe o resultado final do nosso df ao utilizar o método `pd.melt()`:

In [62]:
pd.melt(table_prob,
       id_vars=['Sex'])

,Sex,variable,value
0,female,1,0.968085
1,male,1,0.368852
2,female,2,0.921053
3,male,2,0.157407
4,female,3,0.500000
5,male,3,0.135447


Ele converteu as distintas colunas de Pclass em uma coluna com a categoria da classe e outra com seu valor. <br>
Para facilitar o entendimento das novas colunas podemos renomea-las:

In [58]:
pd.melt(table_prob,
       id_vars=['Sex'],
       var_name='Class_melt',
       value_name='porc_of_survived')

,Sex,Class_melt,porc_of_survived
0,female,1,0.968085
1,male,1,0.368852
2,female,2,0.921053
3,male,2,0.157407
4,female,3,0.500000
5,male,3,0.135447


## Transformação de dados

### pd.cut()
O método `pd.cut()` ordena os dados, separa em bins e computa qual grupo cada linha do df pertence. O `pd.cut()` escolherá os bins para serem espaçados uniformemente de acordo com os próprios valores e não com a frequência desses valores.  <br>
Ele é muito utilizado para transformar variáveis contínuas em categóricas. Por exemplo, podemos converter o valor númerico da idade em grupos de criança, jovem, adulto e idoso.
<br><br>
<a href='https://pandas.pydata.org/docs/reference/api/pandas.cut.html'>Parâmetros:</a> <br>
pandas.cut(x, bins, right=True, labels=None, retbins=False, precision=3, include_lowest=False, duplicates='raise', ordered=True)

Ao informar quantidade de grupos o pd.cut() escolhe os bins com o mesmo tamanho de janela :


In [82]:
df['cut_bins'] = pd.cut(df.Age, 4)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,cut_bins,cut_classes
0,0,3,male,22.0,1,0,7.2500,NaN,S,"(20.315, 40.21]",adultos
1,1,1,female,38.0,1,0,71.2833,C85,C,"(20.315, 40.21]",adultos
2,1,3,female,26.0,0,0,7.9250,NaN,S,"(20.315, 40.21]",adultos
3,1,1,female,35.0,1,0,53.1000,C123,S,"(20.315, 40.21]",adultos
4,0,3,male,35.0,0,0,8.0500,NaN,S,"(20.315, 40.21]",adultos


Podemos passar o nome dos grupos e transformar a variável numérica diretamente em categórica


In [78]:
df['cut_classes'] = pd.cut(df.Age, 4, labels=["jovens", "adultos", "meia-idade", "idosos"])
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,cut_bins,cut_classes
0,0,3,male,22.0,1,0,7.2500,NaN,S,"(20.315, 40.21]",adultos
1,1,1,female,38.0,1,0,71.2833,C85,C,"(20.315, 40.21]",adultos
2,1,3,female,26.0,0,0,7.9250,NaN,S,"(20.315, 40.21]",adultos
3,1,1,female,35.0,1,0,53.1000,C123,S,"(20.315, 40.21]",adultos
4,0,3,male,35.0,0,0,8.0500,NaN,S,"(20.315, 40.21]",adultos


In [90]:
df.cut_bins.unique()

[(20.315, 40.21], NaN, (40.21, 60.105], (0.34, 20.315], (60.105, 80.0]]
Categories (4, interval[float64]): [(0.34, 20.315] < (20.315, 40.21] < (40.21, 60.105] < (60.105, 80.0]]

In [86]:
df.cut_classes.value_counts()

adultos       385
jovens        179
meia-idade    128
idosos         22
Name: cut_classes, dtype: int64

In [87]:
df.cut_bins.value_counts()

(20.315, 40.21]    385
(0.34, 20.315]     179
(40.21, 60.105]    128
(60.105, 80.0]      22
Name: cut_bins, dtype: int64

Também podemos passar uma lista com os valores de início e fim dos bins:

In [81]:
pd.cut(df.Age, [0,20,60,80]).unique()

[(20.0, 60.0], NaN, (0.0, 20.0], (60.0, 80.0]]
Categories (3, interval[int64]): [(0, 20] < (20, 60] < (60, 80]]

### pd.qcut()
O `pd.qcut()` é utilizado quando queremos discretizar nossos dados em quantis. Ao informar quantidade de grupos o `pd.qcut()` escolhe os bins tal que tenhamos a mesma quantidade de valores em cada grupo.

#### `pd.qcut()` x `pd.qcut()`
   * O comando `pd.cut()` cria **caixas equidistantes**, mas a **frequência** das amostras é **desigual** em cada caixa
   * O comando `pd.qcut()` cria **caixas de tamanhos desiguais**, mas a **frequência** das amostras é **igual** em cada caixa.

<br>
Parâmetros:<br>
pandas.qcut(x, q, labels=None, retbins=False, precision=3, duplicates='raise')

In [93]:
pd.cut(df.Age, 4).value_counts()

(20.315, 40.21]    385
(0.34, 20.315]     179
(40.21, 60.105]    128
(60.105, 80.0]      22
Name: Age, dtype: int64

In [31]:
pd.qcut(df.Age, 4).value_counts()

(20.125, 28.0]     183
(0.419, 20.125]    179
(38.0, 80.0]       177
(28.0, 38.0]       175
Name: Age, dtype: int64

In [32]:
pd.qcut(df.Age, 4).value_counts()/df.Age.notnull().sum()

(20.125, 28.0]     0.256303
(0.419, 20.125]    0.250700
(38.0, 80.0]       0.247899
(28.0, 38.0]       0.245098
Name: Age, dtype: float64

<a href='https://towardsdatascience.com/discretisation-using-decision-trees-21910483fa4b'>Discretização utilizando decision trees</a>

### pd.get_dummies()

#### variáveis categóricas
Variáveis categóricas são aquelas que representam grupos ou classes dentro dos nossos dados. Elas podem ser de dois tipos:
* ordinais: possuem uma ordem que tem um sentido. Por exemplo, em rendimentos poderíamos ter: classe alta > classe média > classe baixa  
* nominais: não possuem uma ordem válida. Por exemplo: sexo e CEP.

<img src="variaveis_categoricas.jpeg" style="width: 500px">

Dummies são quaisquer variáveis cujos valores são 1 ou 0 para cada observação. O método `pd.get_dummies()` converte as variáveis categóricas em numéricas separando cada categoria em uma coluna única.
<br>
<br>
<a href="https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html">Parâmetros:</a> <br>
pandas.get_dummies(data, prefix=None, prefix_sep='_', dummy_na=False, columns=None, sparse=False, drop_first=False, dtype=None)

In [96]:
pd.get_dummies(df, columns=['Sex', 'cut_classes'], drop_first=True)

,Survived,Pclass,Age,SibSp,Parch,Fare,Cabin,Embarked,cut_bins,Sex_male,cut_classes_adultos,cut_classes_meia-idade,cut_classes_idosos
0,0,3,22.0,1,0,7.2500,NaN,S,"(20.315, 40.21]",1,1,0,0
1,1,1,38.0,1,0,71.2833,C85,C,"(20.315, 40.21]",0,1,0,0
2,1,3,26.0,0,0,7.9250,NaN,S,"(20.315, 40.21]",0,1,0,0
3,1,1,35.0,1,0,53.1000,C123,S,"(20.315, 40.21]",0,1,0,0
4,0,3,35.0,0,0,8.0500,NaN,S,"(20.315, 40.21]",1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,27.0,0,0,13.0000,NaN,S,"(20.315, 40.21]",1,1,0,0
887,1,1,19.0,0,0,30.0000,B42,S,"(0.34, 20.315]",0,0,0,0
888,0,3,NaN,1,2,23.4500,NaN,S,NaN,0,0,0,0
889,1,1,26.0,0,0,30.0000,C148,C,"(20.315, 40.21]",1,1,0,0


In [100]:
pd.get_dummies(pd.cut(df.Age, 4))

,"(0.34, 20.315]","(20.315, 40.21]","(40.21, 60.105]","(60.105, 80.0]"
0,0,1,0,0
1,0,1,0,0
2,0,1,0,0
3,0,1,0,0
4,0,1,0,0
...,...,...,...,...
886,0,1,0,0
887,1,0,0,0
888,0,0,0,0
889,0,1,0,0


## Multi-index

In [102]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,cut_bins,cut_classes
0,0,3,male,22.0,1,0,7.2500,NaN,S,"(20.315, 40.21]",adultos
1,1,1,female,38.0,1,0,71.2833,C85,C,"(20.315, 40.21]",adultos
2,1,3,female,26.0,0,0,7.9250,NaN,S,"(20.315, 40.21]",adultos
3,1,1,female,35.0,1,0,53.1000,C123,S,"(20.315, 40.21]",adultos
4,0,3,male,35.0,0,0,8.0500,NaN,S,"(20.315, 40.21]",adultos


Para setar indexes use o método `set_index()` indicando quais as colunas quer utilizar como uma lista.

In [126]:
df_row_index = df.set_index(["Pclass", 'Sex'])
df_row_index

,,Survived,Age,SibSp,Parch,Fare,Cabin,Embarked,cut_bins,cut_classes
Pclass,Sex,,,,,,,,,
3,male,0,22.0,1,0,7.2500,NaN,S,"(20.315, 40.21]",adultos
1,female,1,38.0,1,0,71.2833,C85,C,"(20.315, 40.21]",adultos
3,female,1,26.0,0,0,7.9250,NaN,S,"(20.315, 40.21]",adultos
1,female,1,35.0,1,0,53.1000,C123,S,"(20.315, 40.21]",adultos
3,male,0,35.0,0,0,8.0500,NaN,S,"(20.315, 40.21]",adultos
...,...,...,...,...,...,...,...,...,...,...
2,male,0,27.0,0,0,13.0000,NaN,S,"(20.315, 40.21]",adultos
1,female,1,19.0,0,0,30.0000,B42,S,"(0.34, 20.315]",jovens
3,female,0,NaN,1,2,23.4500,NaN,S,NaN,NaN


Para resetar apenas alguns index utilize o método `reset_index([list_columns])` passando uma lista com o nome das colunas como argumento.

In [127]:
df_row_index.reset_index(['Sex'])

,Sex,Survived,Age,SibSp,Parch,Fare,Cabin,Embarked,cut_bins,cut_classes
Pclass,,,,,,,,,,
3,male,0,22.0,1,0,7.2500,NaN,S,"(20.315, 40.21]",adultos
1,female,1,38.0,1,0,71.2833,C85,C,"(20.315, 40.21]",adultos
3,female,1,26.0,0,0,7.9250,NaN,S,"(20.315, 40.21]",adultos
1,female,1,35.0,1,0,53.1000,C123,S,"(20.315, 40.21]",adultos
3,male,0,35.0,0,0,8.0500,NaN,S,"(20.315, 40.21]",adultos
...,...,...,...,...,...,...,...,...,...,...
2,male,0,27.0,0,0,13.0000,NaN,S,"(20.315, 40.21]",adultos
1,female,1,19.0,0,0,30.0000,B42,S,"(0.34, 20.315]",jovens
3,female,0,NaN,1,2,23.4500,NaN,S,NaN,NaN


In [129]:
df_row_index

,,Survived,Age,SibSp,Parch,Fare,Cabin,Embarked,cut_bins,cut_classes
Pclass,Sex,,,,,,,,,
3,male,0,22.0,1,0,7.2500,NaN,S,"(20.315, 40.21]",adultos
1,female,1,38.0,1,0,71.2833,C85,C,"(20.315, 40.21]",adultos
3,female,1,26.0,0,0,7.9250,NaN,S,"(20.315, 40.21]",adultos
1,female,1,35.0,1,0,53.1000,C123,S,"(20.315, 40.21]",adultos
3,male,0,35.0,0,0,8.0500,NaN,S,"(20.315, 40.21]",adultos
...,...,...,...,...,...,...,...,...,...,...
2,male,0,27.0,0,0,13.0000,NaN,S,"(20.315, 40.21]",adultos
1,female,1,19.0,0,0,30.0000,B42,S,"(0.34, 20.315]",jovens
3,female,0,NaN,1,2,23.4500,NaN,S,NaN,NaN


### multi-index nas colunas

In [111]:
table

Survived              
Pclass        1   2    3  All
Sex                          
female       91  70   72  233
male         45  17   47  109
All         136  87  119  342

Acessando colunas:

In [109]:
table.columns

MultiIndex([('Survived',     1),
            ('Survived',     2),
            ('Survived',     3),
            ('Survived', 'All')],
           names=[None, 'Pclass'])

Como acessar uma coluna:

In [115]:
table[('Survived', 1)]

Sex
female     91
male       45
All       136
Name: (Survived, 1), dtype: int64

Slice usando multi-index

In [116]:
table.loc[:, ('Survived', 1):('Survived', 3)]

Survived         
Pclass        1   2    3
Sex                     
female       91  70   72
male         45  17   47
All         136  87  119

Para obter o nome das colunas de cada nível hierárquico

In [140]:
table.columns.get_level_values(0)

Index(['Survived', 'Survived', 'Survived', 'Survived'], dtype='object')

In [136]:
table.columns.get_level_values(1)

Index([1, 2, 3, 'All'], dtype='object', name='Pclass')

In [139]:
nivel_0 = table.columns.get_level_values(0)
nivel_1 = table.columns.get_level_values(1)

[j + '_' + str(nivel_1[i]) for i, j in enumerate(nivel_0)]

['Survived_1', 'Survived_2', 'Survived_3', 'Survived_All']

## Exercícios

1. Baixe os dados de consumo de bebidas por país do <a href="https://www.kaggle.com/justmarkham/alcohol-consumption-by-country">kaggle</a> faça uma análise das informações utilizando os métodos que você já conhece e depois responda:

In [150]:
drinks = pd.read_csv("data/drinks.csv")
drinks

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,continent
0,Afghanistan,0,0,0,0.0,Asia
1,Albania,89,132,54,4.9,Europe
2,Algeria,25,0,14,0.7,Africa
3,Andorra,245,138,312,12.4,Europe
4,Angola,217,57,45,5.9,Africa
...,...,...,...,...,...,...
188,Venezuela,333,100,3,7.7,South America
189,Vietnam,111,2,1,2.0,Asia
190,Yemen,6,0,0,0.1,Asia
191,Zambia,32,19,4,2.5,Africa


a. Encontre qual a bebida mais consumida em cada um dos países e a quantidade.

In [205]:
df1 = drinks.set_index(['country','continent']).max(axis=1).reset_index()
df1

,country,continent,0
0,Afghanistan,Asia,0.0
1,Albania,Europe,132.0
2,Algeria,Africa,25.0
3,Andorra,Europe,312.0
4,Angola,Africa,217.0
...,...,...,...
188,Venezuela,South America,333.0
189,Vietnam,Asia,111.0
190,Yemen,Asia,6.0
191,Zambia,Africa,32.0


In [206]:
df2 = drinks.set_index(['country','continent']).idxmax(axis=1).reset_index()
df2

,country,continent,0
0,Afghanistan,Asia,beer_servings
1,Albania,Europe,spirit_servings
2,Algeria,Africa,beer_servings
3,Andorra,Europe,wine_servings
4,Angola,Africa,beer_servings
...,...,...,...
188,Venezuela,South America,beer_servings
189,Vietnam,Asia,beer_servings
190,Yemen,Asia,beer_servings
191,Zambia,Africa,beer_servings


In [207]:
df2.merge(df1, on=['country', 'continent'])

,country,continent,0_x,0_y
0,Afghanistan,Asia,beer_servings,0.0
1,Albania,Europe,spirit_servings,132.0
2,Algeria,Africa,beer_servings,25.0
3,Andorra,Europe,wine_servings,312.0
4,Angola,Africa,beer_servings,217.0
...,...,...,...,...
188,Venezuela,South America,beer_servings,333.0
189,Vietnam,Asia,beer_servings,111.0
190,Yemen,Asia,beer_servings,6.0
191,Zambia,Africa,beer_servings,32.0


b. Crie um df cujas bebidas estejam agrupadas em uma mesma coluna.

In [153]:
drinks_melt = drinks.melt(id_vars=['country','continent'], value_name='volume', var_name='drink')
drinks_melt

,country,continent,drink,volume
0,Afghanistan,Asia,beer_servings,0.0
1,Albania,Europe,beer_servings,89.0
2,Algeria,Africa,beer_servings,25.0
3,Andorra,Europe,beer_servings,245.0
4,Angola,Africa,beer_servings,217.0
...,...,...,...,...
767,Venezuela,South America,total_litres_of_pure_alcohol,7.7
768,Vietnam,Asia,total_litres_of_pure_alcohol,2.0
769,Yemen,Asia,total_litres_of_pure_alcohol,0.1
770,Zambia,Africa,total_litres_of_pure_alcohol,2.5


c. Utilizando esse novo df, encontre qual a bebida mais consumida por país e a quantidade.

In [188]:
drinks_melt.loc[drinks_melt.groupby('country').idxmax().volume]

,country,continent,drink,volume
0,Afghanistan,Asia,beer_servings,0.0
194,Albania,Europe,spirit_servings,132.0
2,Algeria,Africa,beer_servings,25.0
389,Andorra,Europe,wine_servings,312.0
4,Angola,Africa,beer_servings,217.0
...,...,...,...,...
188,Venezuela,South America,beer_servings,333.0
189,Vietnam,Asia,beer_servings,111.0
190,Yemen,Asia,beer_servings,6.0
191,Zambia,Africa,beer_servings,32.0


2. Considere os dados de preço de fechamento e volume das ações que estão dentro de "data/stocks.csv". <br>
a. Escolha um método de python ensinado na aula de hoje para obter um dataframe cujas linhas são os códigos das ações e as colunas são as datas.

In [208]:
stocks = pd.read_csv("data/stocks.csv")

In [211]:
stocks

,Date,Close,Volume,Symbol
0,2016-10-03,31.50,14070500,CSCO
1,2016-10-03,112.52,21701800,AAPL
2,2016-10-03,57.42,19189500,MSFT
3,2016-10-04,113.00,29736800,AAPL
4,2016-10-04,57.24,20085900,MSFT
5,2016-10-04,31.35,18460400,CSCO
6,2016-10-05,57.64,16726400,MSFT
7,2016-10-05,31.59,11808600,CSCO
8,2016-10-05,113.05,21453100,AAPL


In [212]:
stocks_pivot = stocks.pivot_table(values='Close', index='Symbol', columns='Date')
stocks_pivot

Date,2016-10-03,2016-10-04,2016-10-05
Symbol,,,
AAPL,112.52,113.00,113.05
CSCO,31.50,31.35,31.59
MSFT,57.42,57.24,57.64


b. Com o df original, converta o código das ações para variáveis dummies.

In [222]:
pd.get_dummies(stocks, columns=['Symbol'], drop_first=True)

,Date,Close,Volume,Symbol_CSCO,Symbol_MSFT
0,2016-10-03,31.50,14070500,1,0
1,2016-10-03,112.52,21701800,0,0
2,2016-10-03,57.42,19189500,0,1
3,2016-10-04,113.00,29736800,0,0
4,2016-10-04,57.24,20085900,0,1
5,2016-10-04,31.35,18460400,1,0
6,2016-10-05,57.64,16726400,0,1
7,2016-10-05,31.59,11808600,1,0
8,2016-10-05,113.05,21453100,0,0


3. Considere os dados do arquivo "german_credit.csv" que contem dados de empréstimos realizados por um banco.<br>
a. Encontre qual a média de empréstimo ("Credit Amount") obtidos considerando o propósito ("Purpose") do empréstimo e o sexo ("Sex").

In [216]:
gc = pd.read_csv('data/german_credit.csv')
gc.head()

,Unnamed: 0,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose
0,0,67,male,2,own,NaN,little,1169,6,radio/TV
1,1,22,female,2,own,little,moderate,5951,48,radio/TV
2,2,49,male,1,own,little,NaN,2096,12,education
3,3,45,male,2,free,little,little,7882,42,furniture/equipment
4,4,53,male,2,free,little,little,4870,24,car


In [218]:
gc.pivot_table(index="Purpose", columns = ['Sex'] , values="Credit amount", aggfunc='mean')

Sex,female,male
Purpose,,
business,3195.421053,4392.525641
car,3369.723404,3922.333333
domestic appliances,1409.833333,1586.166667
education,2134.041667,3390.171429
furniture/equipment,2774.729730,3269.112150
radio/TV,2400.517647,2525.635897
repairs,2126.400000,2905.058824
vacation/others,11653.666667,7061.222222


In [221]:
gc.groupby(['Purpose', 'Sex'])[['Credit amount']].mean()

Credit amount
Purpose             Sex                  
business            female    3195.421053
                    male      4392.525641
car                 female    3369.723404
                    male      3922.333333
domestic appliances female    1409.833333
                    male      1586.166667
education           female    2134.041667
                    male      3390.171429
furniture/equipment female    2774.729730
                    male      3269.112150
radio/TV            female    2400.517647
                    male      2525.635897
repairs             female    2126.400000
                    male      2905.058824
vacation/others     female   11653.666667
                    male      7061.222222

b. Converta as variáveis categóricas em numéricas.

In [226]:
gc.dtypes

Unnamed: 0           int64
Age                  int64
Sex                 object
Job                  int64
Housing             object
Saving accounts     object
Checking account    object
Credit amount        int64
Duration             int64
Purpose             object
dtype: object

In [232]:
gc['Checking account'].unique()

array(['little', 'moderate', nan, 'rich'], dtype=object)

In [234]:
pd.get_dummies(gc, columns=['Saving accounts', 'Sex', 'Checking account', 'Purpose'], drop_first=True)

,Unnamed: 0,Age,Job,Housing,Credit amount,Duration,Saving accounts_moderate,Saving accounts_quite rich,Saving accounts_rich,Sex_male,Checking account_moderate,Checking account_rich,Purpose_car,Purpose_domestic appliances,Purpose_education,Purpose_furniture/equipment,Purpose_radio/TV,Purpose_repairs,Purpose_vacation/others
0,0,67,2,own,1169,6,0,0,0,1,0,0,0,0,0,0,1,0,0
1,1,22,2,own,5951,48,0,0,0,0,1,0,0,0,0,0,1,0,0
2,2,49,1,own,2096,12,0,0,0,1,0,0,0,0,1,0,0,0,0
3,3,45,2,free,7882,42,0,0,0,1,0,0,0,0,0,1,0,0,0
4,4,53,2,free,4870,24,0,0,0,1,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,31,1,own,1736,12,0,0,0,0,0,0,0,0,0,1,0,0,0
996,996,40,3,own,3857,30,0,0,0,1,0,0,1,0,0,0,0,0,0
997,997,38,2,own,804,12,0,0,0,1,0,0,0,0,0,0,1,0,0
998,998,23,2,free,1845,45,0,0,0,1,0,0,0,0,0,0,1,0,0


4. Considere o dataset (fake) com testes de aceleração para três carros distintos. Utilize um dos métodos ensinados em aula para criar uma única coluna com os valores das datas e outra com os valores das acelerações.

In [223]:
s = 'Carro A'
x = 'Carro B'
three = 'Carro C'

s_data = [s, 2.5, 2.51, 2.54]
x_data = [x, 2.92, 2.91, 2.93]
three_data = [three, 3.33, 3.31, 3.35]

data = [s_data, x_data, three_data] 
car = pd.DataFrame(data, columns=['car_model', 'Sept 1 9am', 'Sept 1 10am', 'Sept 1 11am'])
car

,car_model,Sept 1 9am,Sept 1 10am,Sept 1 11am
0,Carro A,2.50,2.51,2.54
1,Carro B,2.92,2.91,2.93
2,Carro C,3.33,3.31,3.35


In [224]:
car_melt = car.melt(id_vars=['car_model'], var_name='date', value_name='0-60mph_in_seconds')
car_melt

,car_model,date,0-60mph_in_seconds
0,Carro A,Sept 1 9am,2.50
1,Carro B,Sept 1 9am,2.92
2,Carro C,Sept 1 9am,3.33
3,Carro A,Sept 1 10am,2.51
4,Carro B,Sept 1 10am,2.91
5,Carro C,Sept 1 10am,3.31
6,Carro A,Sept 1 11am,2.54
7,Carro B,Sept 1 11am,2.93
8,Carro C,Sept 1 11am,3.35


## Referências:
pd.melt(): <br>
https://towardsdatascience.com/shape-tables-like-jelly-with-pandas-melt-and-pivot-f2e13e666d6 <br>
https://pub.towardsai.net/understanding-pandas-melt-pd-melt-362954f8c125